# Route Planner - Interactive Dashboard

This is the code for creating an interactive dashboard for our project. This version allows the user to select how close the destination should be.


## Requirements

packages - geopy, geopandas, numpy, utm, folium, geog, json, shapely, plotly_express, matplotlib, osmnx, networkx

Additional packages used - warnings (to ignore the ShapelyDeprecationWarning)

This requires to install mljar-mercury inorder to display the dashboard.

#### pip install mljar-mercury ####

After installing go to the directory that contains the .ipynb file and run the below command in the terminal:

mercury watch dash.ipynb

In [28]:
## introduce the default values for all the inputs to be used in mercury ##
place = "Mile End"
visit = "cafe"
mode = "drive"
time = 4
close = [1,5]

In [29]:
## Import all the packages used for the project ##

from geopy.geocoders import Nominatim
import geopandas as gpd
import numpy as np
import utm
import folium
from IPython.display import Markdown, display
import json
import geog
import shapely.geometry
import plotly_express as px
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import osmnx as ox
import networkx as nx
ox.settings.log_console=True
ox.settings.use_cache=True

## Introduce functions required for the process

This section includes all the functions required for processing the data and filtering

In [30]:

## To convert latitude/longitude to UTM coordinates
def utm_dist(lat,long):
    #convert lat/long to utm
    u = utm.from_latlon(lat, long)
    #return UTM coordinates
    return (u[0],u[1])

def printmd(string):
    display(Markdown(string))   

## Function to return the maximum distance that can be travelled based on the mode of travel and time##
def dist_fn(mode, time):
    #create a dictionary with average speed for each mode of travel
    speed_dict = {'walk':5, 'drive':50, 'bike':19}
    #convert the speed to m/s
    sp = speed_dict[mode]/3.6
    #find distance from given speed and time
    dist = sp * time
    #return distance
    return dist

## Function to find the latitude/longitude of the input location ##
def latlong(place):
    #Initialize Nominatim
    geolocator = Nominatim(user_agent="MyApp")
    #Get the geo-coordinates of current location
    coordinates = geolocator.geocode(place)
    #Latitude of the place
    lat=float(coordinates.latitude)
    #Longitude of the place
    long=float(coordinates.longitude)
    orig = (lat,long)
    return orig

## Function to import the dataset based on the type of place to visit ##
def data(interest):
    #dictionary with place of visit as keys and columns to be extracted as values
    dict1={'Tourist_spots':['addr:postcode','name','opening_hours','tourism','wheelchair:description','geometry'],\
           'pub':['id','addr:postcode','name','opening_hours','website','geometry'],\
           'cafe':['id','addr:postcode','name','opening_hours','outdoor_seating','geometry'],\
           'restaurant':['id','addr:postcode','addr:street','name','opening_hours','phone','geometry']}
    #open the geojson file for the corresponding place of interest   
    df = gpd.read_file(interest+'.geojson')
    #remove the rows where 'name' is NAN
    df = df.dropna(axis=0, subset=['name'])
    #Extract columns based on the point of interest - from dict1
    df=df[dict1[interest]]
    #return the dataframe
    return df

def path(G, orig, dest):
    #find the nearest nodes for the current location and the target
    orig_node, dest_node = ox.nearest_nodes(G,X=[orig[0], dest[0]],\
                            Y=[orig[1], dest[1]])
    #if there is a path that can be found between the two nodes in G
    if nx.has_path(G, orig_node, dest_node):
        #opt_route is the shortest path between orig_node and dest_node where travel_time was considered as the edge weight
        opt_route = nx.shortest_path(G, orig_node, dest_node, weight='travel_time')
        return opt_route
    #if there is no such route
    else:
        #return 0
        return 0
    
## Function for geofencing - filter the data points inside the custom polygon ##
def filtr(df, mask):
    #check whether the points are inside or outside of the polygon
    mask_geofence = df.within(mask)
    #save the value as a new column geofence
    df.loc[:,'geofence'] = mask_geofence
    #drop the rows where geofence is False
    df = df.loc[df['geofence'] == True]
    #drop the column geofence and return the dataframe
    return df.drop(['geofence'], axis=1)

def find_time(G, route, att):
    #sum all the travel times on edges in the path together to get the total time
    time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route, att)))
    return time

In [31]:
## print the heading for the output ##

head = f'Destinations in the closeness range {close[0]} to {close[1]}'
Markdown(f'<h1><center><strong> {head} </strong></center></h1>')

<h1><center><strong> Destinations in the closeness range 1 to 5 </strong></center></h1>

In [32]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")
#Get the geo-coordinates of current location
coordinates = geolocator.geocode(place)
#Latitude of the current location
lat=float(coordinates.latitude)
#Longitude of the current location
long=float(coordinates.longitude)
#Latitude/Longitude coordinate
orig = (lat,long)
#convert time to sec
time = time * 60

## Import data and create polygon for geofencing

Here, we import the file based on the point of interest. Maximum distance that a person can travel from the input place by the given mode of travel is found and use this distance to find a circle with centre as the input place and radius as the distance. Plot the figure with the circle and the datapoints.

In [33]:
#import the dataset based on the type of place to visit with specific columns
df = data(visit)
# the maximum distance that can be travelled based on the mode of travel and time
dist = dist_fn(mode, time)

#Create Point geometric object - `Point' takes positional coordinate values or point tuple parameters
p = shapely.geometry.Point((long,lat))
n_points = 500
#produce 500 (n_points) evenly spaced numbers between 0 and 360
angles = np.linspace(0, 360, n_points)
# buffer a single point by passing in multiple angles (values of `angles).
circle = geog.propagate(p, angles, dist)

#Save the points as a geojson file to construct the circle required for geofencing
with open('Circle.geojson', 'w') as f:
    f.write(json.dumps(shapely.geometry.mapping(shapely.geometry.Polygon(circle))))

#Load the geojson file Circle.geojson to a geodataframe
Circle = gpd.read_file("Circle.geojson")

## Filter data inside the boundary

In [34]:
#create a mask using the coordinates
mask = (Circle.loc[0, 'geometry'])

#apply filtr function to filter the data points which is within the circle
df = filtr(df,mask)

#save the latitude/longitude values in appropriate columns
df["long"] = df['geometry'].map(lambda p: p.x)
df["lat"] = df['geometry'].map(lambda p: p.y)

## Creating projected graph

In [35]:
## CREATING GRAPH ##
#create a graph with orig as the centre,  distance - dist for the selected mode of travel
G = ox.graph_from_point(center_point=orig, dist=dist, network_type=mode)
#Add speed as edge attribute
G = ox.speed.add_edge_speeds(G)
#Add travel time as edge attribute
G = ox.speed.add_edge_travel_times(G)
#Project the graph to utm for better accuracy
G_proj = ox.project_graph(G)

## Finding shortest path and travel time

In [36]:
#convert the latitude/longitude values of the current location into utm coordinates
orig_utm = utm_dist(lat,long)
# convert the latitude/longitude of all the data points into utm coordinates
df['loc'] = df['geometry'].map(lambda p: utm_dist(p.y,p.x))

#FINDING SHORTEST ROUTE AND TRAVEL TIME
#find route using path function
df['route'] = df['loc'].map(lambda p: path(G_proj, orig_utm, p))
#remove the rows where value of 'route' is 0
df=df[df['route'] !=0]
#use function find_time to find the travel time for the corresponding shortest path
df['time'] = df['route'].map(lambda p: find_time(G_proj, p,"travel_time"))

## Filter points where travel time less than maximum time

In [37]:
## FILTERING POINTS ##

#take the minimum travel time
min_time = df['time'].min()
#check if the minimum travel is less than or equal to the time
if min_time<=time:
    #remove rows where the time is greater than or equal to the given time
    rslt_df = df[df['time'] < time]
else:
    print("Cannot find any destination that can be reached within the specified time.")
    # select the row with minimum time
    rslt_df=df[df['time']==min_time]

## Selecting the destinations within the selected range

In [38]:
#The start and end values for slicing the dataframe
a=close[0]-1
b = close[1]
#Slice the dataframe based on the input range
rslt_df=rslt_df[a:b]
#List of all possible target locations
option_list = rslt_df['name'].tolist()

## Output the choice list ##
choice = ','.join(option_list)
out_list = f'Destinations based on your choices are {choice} \n \n'
print(out_list)

Destinations based on your choices are Cafe Donatella,Hiland,Casablanca Cafe / Alauddin Sweets,Mighty Bite,Gallery Café 
 



## For all the destinations inside the selected range, show the shortest route on a map

In [39]:
#for each element in the option_list
for dest_name in option_list:
    #assign the corresponding postcode to the variable `potcode'
    postcode = rslt_df.loc[rslt_df['name']==dest_name, 'addr:postcode'].iloc[0]
    #assign the opening hours to the variable `openhrs'
    openhrs = rslt_df.loc[rslt_df['name']==dest_name, 'opening_hours'].iloc[0]
    
    # if there is opening hours given in the data
    if openhrs != None:
        # define str1
        str1 = 'opening_hours: ' + openhrs
    else:
        str1 = ''
    
    # concatenate destination name, postcode and opening hours for popup
    pop = f'Name: {dest_name}, postcode:{postcode}' + str1
    
    try:
        # save the entry in `route' column corresponding to the destination
        route = rslt_df.loc[rslt_df['name']==dest_name, 'route'].iloc[0]
        # save the name of the destination
        target = rslt_df.loc[rslt_df['name']==dest_name, 'loc'].iloc[0]
        #plot the shortest route on the map
        route_map = ox.plot_route_folium(G, route, route_color = 'red')
        #Make a marker representing the origin and add it to the map
        folium.Marker([lat, long], popup=place).add_to(route_map)
        #create a marker representing the destination and add it to the map
        folium.Marker([rslt_df[rslt_df['name']==dest_name]['lat'], rslt_df[rslt_df['name']==dest_name]['long']], \
              popup=pop, icon=folium.Icon(color='red') ).add_to(route_map)
        
        print("The shortest route from {org} to {des}".format(org=place, des=dest_name))
        #display route_map
        display(route_map)
        
    except ValueError:
        continue
    
    print("\n \n")

The shortest route from Mile End to Cafe Donatella



 

The shortest route from Mile End to Hiland



 

The shortest route from Mile End to Casablanca Cafe / Alauddin Sweets



 

The shortest route from Mile End to Mighty Bite



 

The shortest route from Mile End to Gallery Café


### References

1. https://networkx.org/documentation/stable/tutorial.html, NETWORKX documentation (Viewed - 10 July, 12:30 PM)
2. https://towardsdatascience.com/the-art-of-geofencing-in-python-e6cc237e172d, The Art of Geofencing in Python (Viewed - 12 July, 2 PM)
3. https://gis.stackexchange.com/questions/268250/generating-polygon-representing-rough-100km-circle-around-latitude-longitude-poi, Generating polygon representing rough 100km circle around latitude/longitude point using Python?, Stack Exchange (Viewed - 13 July, 11:30 AM)
4. https://osmnx.readthedocs.io/en/stable/, OSMnx documentation (Viewed - 10 July, 3 PM)
5. https://medium.com/datasciencearth/map-visualization-with-folium-d1403771717 , Map Visualization with Folium (Viewed - 19 July, 12:30 AM)
6. https://mercury-docs.readthedocs.io/en/latest/interactive-slides/, Mercury documentation (Viewed - 29 July, 12:30 AM)